In [1]:
# Add testing url body text to csv to train a model:


In [2]:
import os, sys, re, time, gc, csv
read_loc = 'dataset/Raw/Read/'
store_loc = 'dataset/Raw/'
save_loc = f'{store_loc}LIWC/'

read_file = 'URL_True' # URL_True, True, Fake

Notebook_start = time.time()

is_URL = False
sufix = 'base'
folders = ['base_body','base_title']
if 'URL' in read_file:
    is_URL = True
    sufix = 'url'
    folders = ['url_body','url_title']
    

from LIWC_Methods import Custom_Methods 


import pandas as pd
import numpy as np


In [3]:
folders = ['url_body','url_title']
#Delete all TF_dicts:(Only run for the true part):
for folder in folders:
    loc = f'{save_loc}{folder}/'
    print('Delete loc: ',loc)
    for f in os.listdir(loc):
        os.remove(os.path.join(loc, f))

Delete loc:  dataset/Raw/LIWC/url_body/
Delete loc:  dataset/Raw/LIWC/url_title/


In [4]:
CM = Custom_Methods()

In [5]:
print(f'{read_loc}.csv')
df_old = pd.read_csv(f'{read_loc}{read_file}.csv',error_bad_lines=False, quotechar='"', thousands=',' ,low_memory=False)

if is_URL:    
    idx = list(df_old[df_old.body != ' '].index)
    df_old = df_old.loc[idx]
else:
    idx = list(df_old[df_old.body != ' '].index)
    df_old = df_old.loc[idx]
    idx = list(df_old[df_old.title != ' '].index)
    df_old = df_old.loc[idx]
    
df_old.sort_values(by=['id'],inplace=True)
df_old.reset_index(drop=True,inplace=True)
df_old['title_url_min'] = ''
df_old['body_url_min'] = ''
df_old['title_url'] = ''
df_old['body_url'] = ''
df_old['title_stem'] = ''
df_old['body_stem'] = ''
df_old['link_cnt'] = 0
df_old['process_state'] = 'Pass'

df_old

dataset/Raw/Read/.csv


,id,news_url,source,title,body,class,title_url_min,body_url_min,title_url,body_url,title_stem,body_stem,link_cnt,process_state
0,True_kag_0,http://www.bbc.com/news/world-us-canada-414191...,bbc.com,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,0,,,,,,,0,Pass
1,True_kag_1,https://www.reuters.com/article/us-filmfestiva...,reuters.com,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",0,,,,,,,0,Pass
2,True_kag_100,https://www.reuters.com/article/us-hitachi-kok...,reuters.com,"Elliott again ups Hitachi Kokusai stake, KKR r...",TOKYO (Reuters) - U.S. hedge fund Elliott Mana...,0,,,,,,,0,Pass
3,True_kag_1000,http://www.bbc.com/news/world-us-canada-415394...,bbc.com,Rapper Nelly arrested over alleged tour bus rape,Image caption A lawyer for the 42-year-old art...,0,,,,,,,0,Pass
4,True_kag_1001,http://abcnews.go.com/Politics/wireStory/senat...,abcnews.go.com,Senate bill to clear obstacles to self-driving...,Legislation that could help usher in a new era...,0,,,,,,,0,Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,True_pol_971,http://www.youtube.com/watch?v=Yj3QAzSWVA4,youtube.com,NaN,NaN,0,,,,,,,0,Pass
2302,True_pol_98,http://thomas.loc.gov/cgi-bin/bdquery/z?d109:H...,thomas.loc.gov,NaN,NaN,0,,,,,,,0,Pass
2303,True_pol_988,http://www.youtube.com/watch?v=THrGMfM3Nqs,youtube.com,NaN,NaN,0,,,,,,,0,Pass
2304,True_pol_99,http://transcripts.cnn.com/TRANSCRIPTS/0706/05...,transcripts.cnn.com,NaN,NaN,0,,,,,,,0,Pass


In [6]:
len(df_old[df_old.id.str.contains('kag')])/len(df_old)

0.7649609713790113

In [7]:
len(df_old[df_old.id.str.contains('gos')])/len(df_old)

0.0

In [8]:
len(df_old[df_old.id.str.contains('pol')])/len(df_old)

0.23503902862098872

In [9]:
df_old.shape

(2306, 14)

In [10]:
i = 0
start = time.time()
last_start = start
save_text = True
    
end_cnt = len(df_old)
# end_cnt = 100


for idx, row in df_old[i:end_cnt].iterrows():
    if idx != -1:
        # print(idx,end=', ')
        gc.collect()
        if idx%250==0:
            last_start = CM.print_progress(idx,row,start,last_start)
        try:
            if is_URL:
                row = CM.scrap_page(row)
            else:
                row = CM.clean_text(row,idx)
            if ("page not found" in row.title_url) or ('page unavailable' in row.title_url) or ("scrapper blocked by website" in row.title_url):
                row.process_state = 'Failed'
                save_text = False
                # print(f'failed to clean text for idx:{idx}, ID: {row["id"]}')

            df = row.copy().to_frame().T
            if pd.notnull(df.body_url[idx]):
                if ((len(df.body_url[idx]) > 25) and (save_text == True)): # for LIWC Processing
                    # print('Text Saved')
                    for part in ['title','body']:
                        col = ['id',f'{part}_url']
                        id_ = df.id[idx]
                        CM.save_LIWC_csv(save_loc,idx,df,part,sufix, id_)                     
                else:
                    # print(f'Text Not Saved for idx:{idx}, ID: {row["id"]}')
                    row.process_state = 'Failed'
                    save_text = True
            else:
                # print(f'df.body_url is null for idx:{idx}, ID: {row["id"]}')
                row.process_state = 'Failed'
                save_text = True
             
            
        except:
            row.process_state = 'Failed'
            save_text = True
            # print(f'failed to clean text for idx:{idx}, ID: {row["id"]}')
            
            
        CM.save_file(idx,df,read_file) 

end = time.time()
print(f'Total time in minutes: {int((end-start)/60)}', end=': ')

Total time:0  , Cycle time: 0.0, RowID:0 Attempting URL number:True_kag_0, URL: http://www.bbc.com/news/world-us-canada-41419190

Total time:11  , Cycle time: 10.7, RowID:250 Attempting URL number:True_kag_1478, URL: http://www.cnn.com/videos/sports/2017/02/24/jameis-winston-ladies-are-supposed-to-be-silent-elementary-students-talk.hln
Total time:22  , Cycle time: 11.3, RowID:500 Attempting URL number:True_kag_1978, URL: https://www.nytimes.com/2017/10/11/theater/michael-friedman-aids-death-theater.html?rref=collection%2Fsectioncollection%2Farts

Total time:33  , Cycle time: 10.9, RowID:750 Attempting URL number:True_kag_2501, URL: https://www.reuters.com/article/us-baseball-mlb-playoffs/highlights-of-wednesdays-mlb-playoff-games-idUSKBN1CH0HE

Total time:44  , Cycle time: 11.5, RowID:1000 Attempting URL number:True_kag_3045, URL: https://www.reuters.com/article/us-fifa-pakistan-ban/pakistan-barred-from-international-football-idUSKBN1CG0W5

Total time:56  , Cycle time: 11.7, RowID:1250

In [11]:
print('Done')

Done


In [12]:
loc = f'{store_loc}URL_True.csv'
print(loc)
df = pd.read_csv(loc)

# Replace the missing link_cnt values
avg_link_cnt = np.rint(np.mean(df.link_cnt))
print(avg_link_cnt)

avg_link_cnt = np.rint(np.mean(df.link_cnt[df.link_cnt>0]))
print(avg_link_cnt)
df.link_cnt = [np.rint(link_cnt) if link_cnt > 0 else avg_link_cnt for link_cnt in df.link_cnt]
df.process_state = ['Pass' if len(str_)> 50 else 'Failed' for str_ in df.body_url]
# save file
df.to_csv(loc,mode='w', header=True, index=False)
df = pd.read_csv(loc)
df.head()

dataset/Raw/URL_True.csv
94.0
95.0


,id,news_url,source,title,body,class,title_url_min,body_url_min,title_url,body_url,title_stem,body_stem,link_cnt,process_state
0,True_kag_0,http://www.bbc.com/news/world-us-canada-414191...,bbc.com,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,0,four ways bob corker skewered donald trump,four ways bob corker skewered donald trump ant...,four ways bob corker skewered donald trump,four ways bob corker skewered donald trump ant...,four way bob corker skewer donald trump,four way bob corker skewer donald trump anthon...,111.0,Pass
1,True_kag_1,https://www.reuters.com/article/us-filmfestiva...,reuters.com,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",0,linklater's war veteran comedy speaks to moder...,"“last flag flying”, a comedy-drama about viet...",linklater s war veteran comedy speaks to moder...,last flag flying a comedy drama about vietnam ...,linklat war veteran comedi speak modern americ...,last flag fli comedi drama vietnam war veteran...,18.0,Pass
2,True_kag_100,https://www.reuters.com/article/us-hitachi-kok...,reuters.com,"Elliott again ups Hitachi Kokusai stake, KKR r...",TOKYO (Reuters) - U.S. hedge fund Elliott Mana...,0,"elliott again ups hitachi kokusai stake, kkr r...",u.s. hedge fund elliott management corp has r...,elliott again ups hitachi kokusai stake kkr ra...,u s hedge fund elliott management corp has rai...,elliott up hitachi kokusai stake kkr rais offe...,u hedg fund elliott manag corp rais stake japa...,16.0,Pass
3,True_kag_1000,http://www.bbc.com/news/world-us-canada-415394...,bbc.com,Rapper Nelly arrested over alleged tour bus rape,Image caption A lawyer for the 42-year-old art...,0,rapper nelly arrested over alleged tour bus rape,rapper nelly arrested over alleged tour bus ra...,rapper nelly arrested over alleged tour bus rape,rapper nelly arrested over alleged tour bus ra...,rapper nelli arrest alleg tour bu rape,rapper nelli arrest alleg tour bu rape publish...,107.0,Pass
4,True_kag_1001,http://abcnews.go.com/Politics/wireStory/senat...,abcnews.go.com,Senate bill to clear obstacles to self-driving...,Legislation that could help usher in a new era...,0,page unavailable,sections sections top stories video live u.s. ...,page unavailable,sections sections top stories video live u s p...,page unavail,section section top stori video live u polit e...,67.0,Pass


In [13]:
len(df[df['process_state']=='Pass'])/len(df)

0.9978317432784042

In [14]:
len(df[df['process_state']=='Pass'])

2301

In [15]:
len(df)

2306

In [16]:
Notebook_end = time.time()
Total_Notebook_Time = round((Notebook_end - Notebook_start)/60.0,)
print(f'This notebook took {Total_Notebook_Time} min to run.')

This notebook took 136 min to run.
